# processing ATAC-seq data with ENCODE pipeline on wynton

SarahFong

20240610

My goal is to implement ENCODE's ATAC-seq pipeline following the guidelines published here:

- Overview - https://www.encodeproject.org/atac-seq/
  
- Github pipeline - https://github.com/ENCODE-DCC/atac-seq-pipeline/tree/master

# setup pipeline for run
- Make a virtual environment
- Follow ENCODE ATAC-seq pipeline install instructions on GitHub
    - i.e. from 'pip install caper' step onwards

0. Create virtual environment

        micromamba create -n atac python=3.9 nextflow nf-core

1. Activate the virtual environment and add channels

        micromamba active atac

A. append channels

        micromamba config append channels defaults --env
        micromamba config append channels bioconda --env
        micromamba config append channels conda-forge --env

B. add cache path to .bash_profile

       cd $HOME
       emacs ./.bash_profile

       # add line
       NXF_SINGULARITY_CACHEDIR = $HOME/nf_cache

       # make dir
        mkdir $HOME/nf_cache

2. Install caper

        pip install caper

3. set wynton background HPC background (SGE: Sun Grid Engine)
    
        caper init sge 

4. clone ATAC-seq pipeline 

        cd /wynton/group/ahituv/bin/pipelines

        git clone https://github.com/ENCODE-DCC/atac-seq-pipeline

5. define INPUT_JSON config as variable in command line

       INPUT_JSON="https://storage.googleapis.com/encode-pipeline-test-samples/encode-atac-seq-pipeline/ENCSR356KRQ_subsampled.json"

6. Test run using Wynton's Apptainer/Singularity linux container

        cd /wynton/group/ahituv/bin/pipelines/atac-seq-pipeline
   
        # check if Singularity works on your machine
        $ singularity exec docker://ubuntu:latest echo hello
        
        # on your local machine (--max-concurrent-tasks 1 is for computers with limited resources)
        $ caper run atac.wdl -i "${INPUT_JSON}" --singularity --max-concurrent-tasks 1
        
        # on HPC, make sure that Caper's conf ~/.caper/default.conf is correctly configured to work with your HPC
        # the following command will submit Caper as a leader job to SLURM with Singularity
        $ caper hpc submit atac.wdl -i "${INPUT_JSON}" --singularity --leader-job-name ANY_GOOD_LEADER_JOB_NAME
        
        # check job ID and status of your leader jobs
        $ caper hpc list
        
        # cancel the leader node to close all of its children jobs
        # If you directly use cluster command like scancel or qdel then
        # child jobs will not be terminated
        $ caper hpc abort [JOB_ID]

## Configure inputs
- Do BEFORE every run

## JSON
- Make JSON file following instructions - https://github.com/ENCODE-DCC/atac-seq-pipeline/blob/master/docs/input_short.md
    - Template.json file - https://github.com/ENCODE-DCC/atac-seq-pipeline/blob/master/example_input_json/template.json 

In [3]:
import os, sys
import glob
import json

In [5]:
# PARAMS
PROJECT_NAME = "WTC11.NGN2.1.PEMF"
PIPELINE_PATH="/wynton/group/ahituv/bin/pipelines/ENCODE-atacseq-pipeline"
DATA_PATH_REP1 = "/wynton/group/ahituv/fongsl/projects/EMF/data/ATAC/WTC11NGN2.1"  # one biological replicate
JSON_FILE = os.path.join(DATA_PATH_REP1, PROJECT_NAME + ".json")

ILLUMINA_ADAPTER = "AGATCGGAAGAGC"

In [ ]:
# get fastq files
os.chdir(DATA_PATH_REP1)
unsorted_fastqs = glob.glob("./**/*.fq.gz", recursive=True)
unsorted_fastqs

In [14]:
# make list of readone and read2 files
R1_SUFFIX, R2_SUFFIX = "L2_1", "L2_2"
R1s, R2s = [],[]
for fq in unsorted_fastqs:
    if R1_SUFFIX in fq:
        R1s.append(os.path.join(DATA_PATH_REP1, fq))
    elif R2_SUFFIX in fq:
        R2s.append(os.path.join(DATA_PATH_REP1, fq))
    else:
        print('check your read1 and read2 filename suffixes!')
R1s, R2s

(['/wynton/group/ahituv/fongsl/projects/EMF/data/ATAC/WTC11NGN2.1/./ctrl1/ctrl1_CKDL240018717-1A_HHC5NDSXC_L2_1.fq.gz',
  '/wynton/group/ahituv/fongsl/projects/EMF/data/ATAC/WTC11NGN2.1/./ctrl3/ctrl3_CKDL240018718-1A_HHC5NDSXC_L2_1.fq.gz',
  '/wynton/group/ahituv/fongsl/projects/EMF/data/ATAC/WTC11NGN2.1/./ctrl4/ctrl4_CKDL240018719-1A_HHC5NDSXC_L2_1.fq.gz',
  '/wynton/group/ahituv/fongsl/projects/EMF/data/ATAC/WTC11NGN2.1/./PEMF1/PEMF1_CKDL240018720-1A_HHC5NDSXC_L2_1.fq.gz',
  '/wynton/group/ahituv/fongsl/projects/EMF/data/ATAC/WTC11NGN2.1/./PEMF2/PEMF2_CKDL240018721-1A_HHC5NDSXC_L2_1.fq.gz',
  '/wynton/group/ahituv/fongsl/projects/EMF/data/ATAC/WTC11NGN2.1/./PEMF3/PEMF3_CKDL240018722-1A_HHC5NDSXC_L2_1.fq.gz',
  '/wynton/group/ahituv/fongsl/projects/EMF/data/ATAC/WTC11NGN2.1/./PEMF4/PEMF4_CKDL240018723-1A_HHC5NDSXC_L2_1.fq.gz'],
 ['/wynton/group/ahituv/fongsl/projects/EMF/data/ATAC/WTC11NGN2.1/./ctrl1/ctrl1_CKDL240018717-1A_HHC5NDSXC_L2_2.fq.gz',
  '/wynton/group/ahituv/fongsl/projects

In [42]:
# dictionary of parameters
json_dict = {
    "atac.title" : PROJECT_NAME,
    "atac.description" : "ATAC-seq on WTC11-NGN2 differentiated neurons +/- 60'PEMF exposure and immediate harvest. Only one biological replicate",

    "atac.pipeline_type" : "atac",
    "atac.align_only" : False,
    "atac.true_rep_only" : False, 

    "atac.genome_tsv" : "https://storage.googleapis.com/encode-pipeline-genome-data/genome_tsv/v4/hg38.tsv",
    "atac.blacklist": "/wynton/group/ahituv/data/dna/black_list/hg38-blacklist.v2.bed.gz",

    "atac.paired_end" : True,  # if ALL replicates are paired-ended.

    "atac.fastqs_rep1_R1" : R1s,
    "atac.fastqs_rep1_R2" : R2s, 

    # ADD MORE BIOLOGICAL REPLICATES
    #"atac.fastqs_rep2_R1" : [ "rep2_R1_L1.fastq.gz", "rep2_R1_L2.fastq.gz" ], # no biological replicates yet!
    #"atac.fastqs_rep2_R2" : [ "rep2_R2_L1.fastq.gz", "rep2_R2_L2.fastq.gz" ],

    "atac.auto_detect_adapter" : True,
    #"atac.adapter" : ILLUMINA_ADAPTER, # illumina adaptors
    #"atac.adapters_rep1_R1" : [ [ILLUMINA_ADAPTER]*len(R1s)],
    #"atac.adapters_rep1_R2" : [ [ILLUMINA_ADAPTER]*len(R2s)],
    
    #"atac.adapters_rep2_R1" : [ "AATTCCGG", "AATTCCGG", "AATTCCGG" ],
    #"atac.adapters_rep2_R2" : [ "AATTCCGG", "AATTCCGG" ],

    "atac.multimapping" : 4
}

In [43]:
# write json file
with open(JSON_FILE, "w") as writer:
    json.dump(json_dict, writer)
    

In [30]:
# run in command line
print(f"INPUT_JSON='{JSON_FILE}'")

INPUT_JSON='/wynton/group/ahituv/fongsl/projects/EMF/data/ATAC/WTC11NGN2.1/WTC11.NGN2.1.PEMF.json'


In [ ]:
#run in command line
os.chdir(PIPELINE_PATH)
" caper run atac.wdl -i "${INPUT_JSON}" --singularity --max-concurrent-tasks 1 --backend sge-pe"""

## Errors!

### error one - java

STDERR=Exception in thread "main" java.lang.BootstrapMethodError: java.lang.UnsupportedClassVersionError: wdl/draft3/parser/WdlParser$Ast has been compiled by a more recent version of the Java Runtime (class file version 55.0), this version of the Java Runtime only recognizes class file versions up to 52.0
	at wdl.draft3.transforms.ast2wdlom.package$.<clinit>(ast2wdlom.scala:16)
	at languages.wdl.draft3.WdlDraft3LanguageFactory.makeWomBundle(WdlDraft3LanguageFactory.scala:64)
	at languages.wdl.draft3.WdlDraft3LanguageFactory$$anon$1.call(WdlDraft3LanguageFactory.scala:79)
	at languages.wdl.draft3.WdlDraft3LanguageFactory$$anon$1.call(WdlDraft3LanguageFactory.scala:78)
	at cromwell.languages.util.ParserCache.$anonfun$retrieveOrCalculate$2(ParserCache.scala:35)
	at scala.Option.getOrElse(Option.scala:201)
	at cromwell.languages.util.ParserCache.retrieveOrCalculate(ParserCache.scala:35)
	at cromwell.languages.util.ParserCache.retrieveOrCalculate$(ParserCache.scala:25)
	at languages.wdl.draft3.WdlDraft3LanguageFactory.retrieveOrCalculate(WdlDraft3LanguageFactory.scala:30)
	at languages.wdl.draft3.WdlDraft3LanguageFactory.$anonfun$getWomBundle$2(WdlDraft3LanguageFactory.scala:86)
	at scala.util.Either.flatMap(Either.scala:352)
	at languages.wdl.draft3.WdlDraft3LanguageFactory.getWomBundle(WdlDraft3LanguageFactory.scala:85)
	at womtool.input.WomGraphMaker$.$anonfun$getBundleAndFactory$1(WomGraphMaker.scala:40)
	at scala.util.Either.flatMap(Either.scala:352)
	at womtool.input.WomGraphMaker$.getBundleAndFactory(WomGraphMaker.scala:31)
	at womtool.input.WomGraphMaker$.fromFiles(WomGraphMaker.scala:47)
	at womtool.validate.Validate$.validate(Validate.scala:26)
	at womtool.WomtoolMain$.dispatchCommand(WomtoolMain.scala:54)
	at womtool.WomtoolMain$.runWomtool(WomtoolMain.scala:161)
	at womtool.WomtoolMain$.delayedEndpoint$womtool$WomtoolMain$1(WomtoolMain.scala:166)
	at womtool.WomtoolMain$delayedInit$body.apply(WomtoolMain.scala:27)
	at scala.Function0.apply$mcV$sp(Function0.scala:39)
	at scala.Function0.apply$mcV$sp$(Function0.scala:39)
	at scala.runtime.AbstractFunction0.apply$mcV$sp(AbstractFunction0.scala:17)
	at scala.App.$anonfun$main$1(App.scala:76)
	at scala.App.$anonfun$main$1$adapted(App.scala:76)
	at scala.collection.IterableOnceOps.foreach(IterableOnce.scala:563)
	at scala.collection.IterableOnceOps.foreach$(IterableOnce.scala:561)
	at scala.collection.AbstractIterable.foreach(Iterable.scala:926)
	at scala.App.main(App.scala:76)
	at scala.App.main$(App.scala:74)
	at womtool.WomtoolMain$.main(WomtoolMain.scala:27)
	at womtool.WomtoolMain.main(WomtoolMain.scala)
Caused by: java.lang.UnsupportedClassVersionError: wdl/draft3/parser/WdlParser$Ast has been compiled by a more recent version of the Java Runtime (class file version 55.0), this version of the Java Runtime only recognizes class file versions up to 52.0
	at java.lang.ClassLoader.defineClass1(Native Method)
	at java.lang.ClassLoader.defineClass(ClassLoader.java:756)
	at java.security.SecureClassLoader.defineClass(SecureClassLoader.java:142)
	at java.net.URLClassLoader.defineClass(URLClassLoader.java:473)
	at java.net.URLClassLoader.access$100(URLClassLoader.java:74)
	at java.net.URLClassLoader$1.run(URLClassLoader.java:369)
	at java.net.URLClassLoader$1.run(URLClassLoader.java:363)
	at java.security.AccessController.doPrivileged(Native Method)
	at java.net.URLClassLoader.findClass(URLClassLoader.java:362)
	at java.lang.ClassLoader.loadClass(ClassLoader.java:418)
	at sun.misc.Launcher$AppClassLoader.loadClass(Launcher.java:352)
	at java.lang.ClassLoader.loadClass(ClassLoader.java:351)
	... 33 more


### try: install openjdk into atac env?  

    micromamba install openjdk

- not getting same error. 

### error two - wrong adaptors

    STDERR=Failed to evaluate input 'adapters_rep1_R1' (reason 1 of 1): No coercion defined from '["AGATCGGAAGAGC","AGATCGGAAGAGC","AGATCGGAAGAGC","AGATCGGAAGAGC","AGATCGGAAGAGC","AGATCGGAAGAGC","AGATCGGAAGAGC"]' of type 'spray.json.JsArray' to 'String'.
    Failed to evaluate input 'adapters_rep1_R2' (reason 1 of 1): No coercion defined from '["AGATCGGAAGAGC","AGATCGGAAGAGC","AGATCGGAAGAGC","AGATCGGAAGAGC","AGATCGGAAGAGC","AGATCGGAAGAGC","AGATCGGAAGAGC"]' of type 'spray.json.JsArray' to 'String'.




### try: automatically detecting adaptors

- in json
  
        "atac.auto_detect_adapter" : True,

  I guess it worked? I'm no longer getting this error

### doesn't recognize atac.black_list dictionary input
Traceback (most recent call last):
  File "/wynton/home/ahituv/fongsl/micromamba/envs/atac/bin/caper", line 13, in <module>
    main()
  File "/wynton/home/ahituv/fongsl/micromamba/envs/atac/lib/python3.9/site-packages/caper/cli.py", line 710, in main
    return runner(parsed_args, nonblocking_server=nonblocking_server)
  File "/wynton/home/ahituv/fongsl/micromamba/envs/atac/lib/python3.9/site-packages/caper/cli.py", line 255, in runner
    subcmd_run(c, args)
  File "/wynton/home/ahituv/fongsl/micromamba/envs/atac/lib/python3.9/site-packages/caper/cli.py", line 385, in subcmd_run
    thread = caper_runner.run(
  File "/wynton/home/ahituv/fongsl/micromamba/envs/atac/lib/python3.9/site-packages/caper/caper_runner.py", line 462, in run
    self._cromwell.validate(wdl=wdl, inputs=inputs, imports=imports)
  File "/wynton/home/ahituv/fongsl/micromamba/envs/atac/lib/python3.9/site-packages/caper/cromwell.py", line 159, in validate
    raise WomtoolValidationFailed(
caper.cromwell.WomtoolValidationFailed: RC=1
STDERR=WARNING: Unexpected input provided: atac.black_list (expected inputs: [atac.adapters_rep6_R2, atac.filter_mem_factor, atac.filter_chrs, atac.conda, atac.pool_ta.col, atac.prom, atac.peak_pooled, atac.peaks_pr2, atac.fastqs_rep2_R2, atac.fastqs_rep8_R2, atac.adapters_rep2_R2, atac.bam2ta_time_hr, atac.conda_macs2, atac.pipeline_type, atac.align_only, atac.fastqs_rep1_R1, atac.xcor_mem_factor, atac.adapters_rep7_R1, atac.mito_chr_name, atac.pseudoreplication_random_seed, atac.call_peak_disk_factor, atac.preseq_picard_java_heap, atac.bam2ta_mem_factor, atac.dup_marker, atac.conda_spp, atac.enable_gc_bias, atac.pool_blacklist.prefix, atac.macs2_signal_track_mem_factor, atac.spr_disk_factor, atac.align_cpu, atac.preseq_mem_factor, atac.paired_ends, atac.roadmap_meta, atac.adapters_rep10_R2, atac.auto_detect_adapter, atac.adapters_rep10_R1, atac.adapters_rep9_R2, atac.xcor_time_hr, atac.ref_fa, atac.fastqs_rep6_R1, atac.adapters_rep5_R2, atac.fraglen_stat_picard_java_heap, atac.adapters_rep1_R1, atac.adapters_rep5_R1, atac.no_dup_removal, atac.adapters_rep3_R1, atac.qc_report.qc_json_ref, atac.idr_thresh, atac.fastqs_rep8_R1, atac.enable_tss_enrich, atac.fastqs_rep7_R1, atac.chrsz, atac.adapters_rep1_R2, atac.subsample_reads, atac.pool_ta_pr1.col, atac.xcor_subsample_reads, atac.peak_ppr2, atac.xcor_disk_factor, atac.enable_preseq, atac.true_rep_only, atac.bams, atac.enable_compare_to_roadmap, atac.fastqs_rep10_R2, atac.reg2map_bed, atac.dnase, atac.fastqs_rep2_R1, atac.fastqs_rep3_R1, atac.preseq.null, atac.enh, atac.read_genome_tsv.null_s, atac.adapters_rep8_R1, atac.fastqs_rep6_R2, atac.filter_cpu, atac.enable_annot_enrich, atac.adapters_rep3_R2, atac.call_peak_time_hr, atac.genome_name, atac.read_len, atac.tss, atac.regex_bfilt_peak_chr_name, atac.fastqs_rep5_R2, atac.adapters_rep4_R1, atac.pool_ta_pr2.col, atac.docker, atac.adapters_rep2_R1, atac.singularity, atac.bowtie2_mito_idx_tar, atac.filter_disk_factor, atac.fastqs_rep4_R2, atac.filter_time_hr, atac.peaks, atac.description, atac.fastqs_rep4_R1, atac.tas, atac.enable_xcor, atac.preseq_disk_factor, atac.title, atac.filter_picard_java_heap, atac.enable_count_signal_track, atac.bam2ta_cpu, atac.fastqs_rep7_R2, atac.blacklist, atac.fastqs_rep9_R2, atac.fastqs_rep10_R1, atac.pval_thresh, atac.jsd_cpu, atac.adapter, atac.macs2_signal_track_disk_factor, atac.call_peak_mem_factor, atac.peaks_pr1, atac.cutadapt_param, atac.align_disk_factor, atac.fastqs_rep1_R2, atac.bam2ta_disk_factor, atac.nodup_bams, atac.call_peak_cpu, atac.adapters_rep6_R1, atac.gc_bias_picard_java_heap, atac.adapters_rep8_R2, atac.jsd_time_hr, atac.cap_num_peak, atac.xcor_cpu, atac.bowtie2_idx_tar, atac.reg2map, atac.adapters_rep7_R2, atac.gensz, atac.adapters_rep4_R2, atac.adapters_rep9_R1, atac.jsd_mem_factor, atac.spr_mem_factor, atac.multimapping, atac.fastqs_rep5_R1, atac.mapq_thresh, atac.blacklist2, atac.fastqs_rep9_R1, atac.genome_tsv, atac.enable_jsd, atac.fastqs_rep3_R2, atac.paired_end, atac.align_time_hr, atac.conda_python2, atac.align_mem_factor, atac.peak_ppr1, atac.macs2_signal_track_time_hr, atac.enable_fraglen_stat, atac.ref_mito_fa, atac.enable_idr, atac.jsd_disk_factor, atac.smooth_win])


### try: black_list -> blacklist
- appears to be a typo

# Alternative strategy: NF-core?

micromamba install nextflow

git clone https://github.com/nf-core/atacseq

make sample sheet (csv)
    
    name,fastq1,fastq2,replicate

## online

In [7]:
NFCORE_PATH = "/wynton/group/ahituv/bin/pipelines/atacseq"

INPUT = os.path.join(DATA_PATH_REP1, "samples.csv")
OUTPUT_DIR = os.path.join(DATA_PATH_REP1, "results")
READLEN = "150"

os.chdir(NFCORE_PATH)

In [9]:
cmd = " ".join(['nextflow run nf-core/atacseq',
                "--input", INPUT, 
                "--outdir", OUTPUT_DIR, 
                "--genome GRCh38", 
                "--read_length", READLEN,
                "-profile singularity", 
                "--narrow_peak",
                "--resume"
               ])
print(cmd)

# before running command, need to load java17
java_cmd = " ".join(["ml load openjdk/17"])
CACHE = "NXF_SINGULARITY_CACHEDIR=/wynton/group/ahituv/bin/pipelines/atacseq/work/singularity"

nextflow run nf-core/atacseq --input /wynton/group/ahituv/fongsl/projects/EMF/data/ATAC/WTC11NGN2.1/samples.csv --outdir /wynton/group/ahituv/fongsl/projects/EMF/data/ATAC/WTC11NGN2.1/results --genome GRCh38 --read_length 150 -profile singularity --narrow_peak --resume


ERROR ~ Error executing process > 'NFCORE_ATACSEQ:ATACSEQ:FASTQ_ALIGN_BWA:BWA_MEM (1)'

Caused by:
  Not a valid S3 file system provider file attribute view: java.nio.file.attribute.BasicWithKeyFileAttributeView



 -- Check '.nextflow.log' file for details

### try increasing java
    ml load openjdk/17

In [31]:
qsub /wynton/home/ahituv/fongsl/EMF/bin/ATAC/nfcore-atac.sh 

SyntaxError: invalid syntax (2814767026.py, line 1)

# update plugins? 

I keep getting the error in my qsub script:
        
        Downloading plugin nf-schema@2.0.0
        ERROR ~ Plugin with id nf-schema not found in any repository

Think I need to update the plugin naming..
see here: https://nextflow-io.github.io/nf-schema/latest/migration_guide/#updating-the-name-and-version-of-the-plugin

Did this: 
1. in command line:

       nextflow plugin install nf-schema@2.0.0
3. in nextflow.config, changed 

       plugins { id 'nf-schema@2.0.0' }

In [51]:
# update one
cmd = """find . -type f -name "*.nf" -exec sed -i -e "s/from 'plugin\/nf-validation'/from 'plugin\/nf-schema'/g" -\
e 's/from "plugin\/nf-validation"/from "plugin\/nf-schema"/g' {} +"""
os.system(cmd)

0

In [54]:
# update 2
cmd = """sed -i -e 's/http:\/\/json-schema.org\/draft-07\/schema/https:\/\/json-schema.org\/draft\/2020-12\/schema/g' -e 's/definitions/defs/g' nextflow_schema.json"""
os.system(cmd)

0

In [53]:
os.chdir(NFCORE_PATH) #+ '/conf')

# running offline
- https://nf-co.re/docs/usage/getting_started/offline

1. create virtualenv

        micromamba create -n env_nf nextflow nf-core

2. activate virualenv

       micromamba actiavte env_nf

3. download nextflow, plugins

        # Download nextflow
        #from https://nf-co.re/docs/usage/getting_started/installation
   
        curl -s https://get.nextflow.io | bash
        chmod +x nextflow
        mv nextflow ~/bin/

        # Download plugins
        # from https://github.com/nextflow-io/nextflow/discussions/4126
        nextflow plugin install nf-schema@2.0.0


       # add to $HOME/.bashrc file:
       export NXF_OFFLINE='true'
   

4. download pipeline with singularity container

       cd /wynton/group/ahituv/bin/pipelines/  # go to working directory
       nf-core download atacseq --singularity  # download atacseq pipeline with singularity container
       gunzip nf-core-atacseq-dev.tar.gz | tar -xzf # unzip and install tar

        # copy the plugins file to the new workflow folder
       cp -r $HOME/.nextflow /wynton/group/ahituv/bin/pipelines/nf-core-atacseq-dev/workflow/
       

   

5. pin plugin in nextflow.config in /wynton/group/ahituv/bin/pipelines/nf-core-atacseq-dev/workflow/:

       // Nextflow plugins
        plugins {
            //id 'nf-validation'  Validation of pipeline parameters and creation of an input channel from a sample sheet // SF changed https://github.com/nextflow-io/nf-validation
            id 'nf-schema@2.0.0' 
        }
   

6. download references
   

In [ ]:
--fasta /wynton/group/ahituv/data/dna/hg38/hg38.fa.gz
--gtf /wynton/group/ahituv/data/dna/hg38/hg38.knownGene.gtf.gz